In [80]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import skimpy
import skimpy.functional as F
import skimpy_3d

In [164]:
def blur(tensor):
    G_x = (1 / 64.0) * skimpy.Tensor.from_list([[[1, 6, 15, 20, 15, 6, 1]]]).to(float)
    G_y = (1 / 64.0) * skimpy.Tensor.from_list([[[1], [6], [15], [20], [15], [6], [1]]]).to(float)
    G_z = (1 / 64.0) * skimpy.Tensor.from_list([[[1]], [[6]], [[15]], [[20]], [[15]], [[6]], [[1]]]).to(float)
    tensor = F.conv_3d(tensor, G_x, padding=(3, 0, 0), fill=0.0)
    tensor = F.conv_3d(tensor, G_y, padding=(0, 3, 0), fill=0.0)  
    tensor = F.conv_3d(tensor, G_z, padding=(0, 0, 3), fill=0.0)
    return tensor

In [165]:
t = skimpy.Tensor(shape=(256, 256, 256), val=0, dtype=float)

In [166]:
t[64:192, 64:192, 64:192] = 1
t[5:120, 70:210, :] = 0
t[86:130, 80:120, 86:170] = 1

In [167]:
%%time
for i in range(5):
    t = blur(t)

Wall time: 1.67 s


In [168]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
config[1] = skimpy_3d.ColorVoxel(0, 200, 200)

In [169]:
%%time
mesh = skimpy_3d.generate_mesh(config, (t > 0.5).to(int)._tensor)

Wall time: 478 ms


In [170]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [171]:
js_mesh = Mesh(
    geometry=BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "normal": BufferAttribute(
                np.array(mesh.normals, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "color": BufferAttribute(
                np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    ),
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [172]:
js_mesh

Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(369760, 3), dtype=f…

In [173]:
%%time
for i in range(2):
    t = sum([
        0.125 * t[0::2, 0::2, 0::2],
        0.125 * t[1::2, 0::2, 0::2],
        0.125 * t[0::2, 1::2, 0::2],
        0.125 * t[1::2, 1::2, 0::2],
        0.125 * t[0::2, 0::2, 1::2],
        0.125 * t[1::2, 0::2, 1::2],
        0.125 * t[0::2, 1::2, 1::2],
        0.125 * t[1::2, 1::2, 1::2],
    ])

Wall time: 48 ms


In [174]:
avg_kernel = skimpy.Tensor.from_list([[[1, 1], [1, 1]], [[1, 1], [1, 1]]]).to(float) / 8
density_lattice = F.conv_3d(
    t,
    avg_kernel,
    padding=1,
)

In [175]:
%%time
mesh = skimpy_3d.marching_cubes(density_lattice._tensor, 0.5)

Wall time: 12 ms


In [176]:
geometry = BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    )

geometry.exec_three_obj_method("computeVertexNormals")

In [185]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(color="blue"),
    position=[0, 0, 0],
)

In [190]:
js_mesh

Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(5416, 3), dtype=flo…

In [179]:
wf_geometry = WireframeGeometry(geometry)

In [180]:
js_lines = LineSegments(
    geometry=wf_geometry,
    material=LineBasicMaterial(color="green"),
    position=[0, 0, 0],
)

In [181]:
js_lines

Preview(child=LineSegments(geometry=WireframeGeometry(geometry=BufferGeometry(attributes={'position': <BufferA…

In [191]:
camera = PerspectiveCamera(position=(64, 64, 64), lookAt=(0,0,0), fov=20)
scene = Scene(children=[js_mesh, camera])

In [194]:
renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)

Renderer(background='white', camera=PerspectiveCamera(fov=20.0, position=(-38.941229764299436, 225.03555780363…